***********************



THIS CREATES THE CSV WITHOUT THE RACE LABEL
**************************

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
df = pd.read_csv('labeling-responses.csv')

# print(df.head())

In [5]:
urls = df['Paste the link of the article read.'].tolist()

wgbh_urls = [url for url in urls if 'wgbh.org' in url]

print(wgbh_urls)
print(len(wgbh_urls))

['https://www.wgbh.org/news/national/2023-03-18/why-car-prices-are-still-so-high-and-why-they-are-unlikely-to-fall-anytime-soon', 'https://www.wgbh.org/news/politics/2023-03-08/boston-city-council-approves-wus-rent-control-and-city-planning-proposals', 'https://www.wgbh.org/news/local/2023-06-06/we-have-a-crisis-leaders-look-to-address-root-of-gun-violence-in-boston', 'https://www.wgbh.org/news/commentary/2023-04-03/it-should-be-so-much-easier-to-cancel-unwanted-subscriptions', 'https://www.wgbh.org/news/local/2023-03-17/basic-black-how-will-bostons-task-force-on-reparations-confront-generational-trauma-caused-by-enslavement', 'https://www.wgbh.org/news/national-news/2023/04/28/airbnb-let-its-workers-live-and-work-anywhere-spoiler-theyre-loving-it', 'https://www.wgbh.org/news/local/2023-07-19/paul-revere-never-owned-slaves-but-he-still-benefited-from-the-slave-trade', 'https://www.wgbh.org/news/local/2023-03-24/10-year-old-tanoshi-inomata-is-spelling-his-way-from-boston-to-d-c', 'https

In [4]:
url = urls[1]
print(url)
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

https://www.wgbh.org/news/national/2023-03-18/why-car-prices-are-still-so-high-and-why-they-are-unlikely-to-fall-anytime-soon


In [27]:
# cleaned up the article body text

article_text = soup.find(class_ = 'articleBody rich-text').text.strip()

article_title = soup.find(class_ = 'heading heading-1').text.strip()

print(article_text)
print(article_title)

It has been nearly three years since auto plants around the world started to shut down because of the pandemic.Yet between the pandemic, an acute shortage of semiconductors and other supply chain snarls, vehicle production has never really returned to normal.And prices? Hoo boy.Both new and used prices have stopped skyrocketing. In fact, both dipped slightly in February.But the average new-vehicle transaction price is still $48,763, according to Kelley Blue Book. Before the pandemic, the average new vehicle sold for $37,876.At an auto show this year, Noah and India Grabisch of Laurel, Md., were looking at new SUVs, which they liked. But an $86,000 price tag for a Chevrolet Suburban?Well ..."It looks nice," India Grabisch said. "But ... no."The used-car market doesn't provide much relief either. Retail used-vehicle prices now average $26,510.And after falling last year, closely watched wholesale prices, a key indicator of where markets are headed, are actually rising again.Why are price

In [9]:
import time


def scrape_wgbh_article(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        article_title = soup.find(class_ = 'heading heading-1')
        if article_title:
            article_title = article_title.text.strip()
        else:
            article_title = "Title not found"

        
        article_body = soup.find(class_ = 'articleBody rich-text')
        if article_body:
            enhancements = article_body.find_all(class_='Enhancement')
            for enhancement in enhancements:
                enhancement.decompose()

            article_text = article_body.get_text(separator="", strip=True)
        else:
            article_text = "Article text not found"


        return {
            'url': url,
            'title': article_title,
            'text': article_text
        }
    
    else:
        print(f"Failed to retrieve content from {url}")
        return None
    
scraped_articles = []

for url in wgbh_urls:
    article_info = scrape_wgbh_article(url)
    if article_info:  # Check if the article info was successfully retrieved
        scraped_articles.append(article_info)
    
    time.sleep(1)  # Pause for 1 second between requests to be respectful to the server

# Example output
for article in scraped_articles[:5]:  # Print first 5 scraped articles to check
    print(article['title'], article['url'])


        
        

Why car prices are still so high — and why they are unlikely to fall anytime soon https://www.wgbh.org/news/national/2023-03-18/why-car-prices-are-still-so-high-and-why-they-are-unlikely-to-fall-anytime-soon
Boston City Council approves Wu's rent control and city planning proposals https://www.wgbh.org/news/politics/2023-03-08/boston-city-council-approves-wus-rent-control-and-city-planning-proposals
‘We have a crisis’: Leaders look to address root of gun violence in Boston https://www.wgbh.org/news/local/2023-06-06/we-have-a-crisis-leaders-look-to-address-root-of-gun-violence-in-boston
It should be so much easier to cancel unwanted subscriptions https://www.wgbh.org/news/commentary/2023-04-03/it-should-be-so-much-easier-to-cancel-unwanted-subscriptions
Basic Black: How will Boston’s task force on reparations confront generational trauma caused by enslavement? https://www.wgbh.org/news/local/2023-03-17/basic-black-how-will-bostons-task-force-on-reparations-confront-generational-trauma-c

In [10]:
import csv 

csv_file_name = 'scraped_wgbh_articles.csv'

headers = ['title', 'text', 'url']

with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    
    # Write the header to the CSV file
    writer.writeheader()
    
    # Write the scraped article data to the CSV file
    for article in scraped_articles:
        writer.writerow(article)

print(f"Data has been written to {csv_file_name}")

Data has been written to scraped_wgbh_articles.csv
